In [1]:
import datetime
import importlib 
import numpy as np
import os

from parcels import Field, FieldSet, ParticleSet,Variable, JITParticle

In [41]:
import OP_functions_shared as OP
import PBDEs_OP_Kernels_shared as PBDE

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [65]:
importlib.reload(PBDE)
importlib.reload(OP)

<module 'OP_functions_shared' from '/ocean/sallen/allen/research/Meopar/ANALYSIS/analysis-vicente/Ocean_Parcels/SHARED/OP_functions_shared.py'>

In [5]:
path = {'out': '/home/sallen/MEOPAR/ANALYSIS/analysis-vicente/Ocean_Parcels/SHARED/',}

In [106]:
start_time = datetime.datetime(2021, 1, 1)
sim_length = 0.05 # days
data_length = max(sim_length, 1)
duration = datetime.timedelta(days=sim_length)
delta_t = 60 # s
release_particles_every = 450 # s

In [115]:
#### CREATING FIELDSETS ####

# Velocities

varlist = ['U', 'V', 'W']
filenames, variables = OP.filename_set(start_time, data_length, varlist)
dimensions = {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw','time': 'time_counter'}
field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True, chunksize='auto')

# Vertical Variables

varlist=['Kz', 'totdepth']
filenames, variables = OP.filename_set(start_time, data_length, varlist)
dimensions = {'lon': 'glamt', 'lat': 'gphit'}
TD = Field.from_netcdf(filenames['totdepth'], variables['totdepth'], dimensions, allow_time_extrapolation=True, chunksize='auto')
field_set.add_field(TD)

# Add Vertical diffusivity coefficient field
dimensions = {'lon': 'glamt', 'lat': 'gphit', 'depth': 'depthw','time': 'time_counter'}
Kz = Field.from_netcdf(filenames['Kz'], variables['Kz'], dimensions, allow_time_extrapolation=True, chunksize='auto')
field_set.add_field(Kz)

# Constants
dt_h = 1 / 3600. 
field_set.add_constant('sinkvel_sewage', 21 * dt_h) # m/hr * dt --> to seconds --> ~ 500 m/d
field_set.add_constant('sinkvel_marine', 10 * dt_h) # m/hr * dt --> to seconds --> ~ 250 m/d

#WARNING MULTIPLED BY 20!!!!!
abso = 0.038 / 86400 *5 # Colloidal/Dissolved → Attached to Marine Particle /s 
deso_s = 3.2 / 86400 *5 # Sewage Particle → Colloidal/Dissolved /s
deso_m = 1.6 / 86400 *5 # Marine Particle → Colloidal/Dissolved /s
field_set.add_constant('abso_probability', 1 - np.exp(-abso * delta_t))
field_set.add_constant('deso_s_probability', 1 - np.exp(-deso_s * delta_t))
field_set.add_constant('deso_m_probability', 1 - np.exp(-deso_m * delta_t))

In [116]:
# IONA OUTFALL COORDINATES
clat = [49.195045]
clon = [-123.301956]
z = 70 # 70 m depth

In [117]:
number_particles = 5
z*np.ones(number_particles)

array([70., 70., 70., 70., 70.])

In [118]:
#Swimming Kernel (play Kernel)
def tu_swim(particle, fieldset, time):
    mbath = 100
#    sink = 20e-3
    swim = 32.5e-3
    dt = 200 # how often they change direction
    kappa = swim**2 * dt
    eff_swim = sqrt(kappa / particle.dt)

    upward = 1.05   # tendency to swim more upward than sink
    zswim = (- particle.dt * (upward-1)*sink + particle.dt * 
             (eff_swim - upward*sink) * math.cos(ParcelsRandom.uniform(0, 2* math.pi)))
    if particle.depth + zswim < 0:
        particle_ddepth = 0
    elif particle.depth + zswim > mbath:
        particle_ddepth = mbath
    else:
        particle_ddepth += zswim

In [119]:
fn = 'particles_out.zarr'
outfile_states = os.path.join(path['out'], fn)

### Particle Class ###

In [120]:
class MPParticle(JITParticle):
    # Status Variable #
    status = Variable('status', initial=-1)
    vvl_factor = Variable('fact', initial=1)
    release_time = Variable('release_time', initial=np.arange(0, release_particles_every*number_particles, release_particles_every))

In [121]:
pset_states = ParticleSet(field_set, pclass=MPParticle, lon=clon*np.ones(number_particles), 
                          depth=z*np.ones(number_particles), lat = clat*np.ones(number_particles))

In [122]:
output_interval = datetime.timedelta(seconds=delta_t)
output_file = pset_states.ParticleFile(name=outfile_states, outputdt=output_interval)
KE = (pset_states.Kernel(PBDE.PBDEs_states) + pset_states.Kernel(PBDE.Sinking) 
      + pset_states.Kernel(PBDE.turb_mix))
# Currently have Sinking and vertical mixing for 5 particles for a bit over an hour.

In [123]:
#KE = PBDE.Sinking #+ pset.Kernel(tu_mix) #if you want to add more kernels use:  KE = tu_mix + NAME_KERNEL1 + ...
pset_states.execute(KE, runtime=duration, dt=delta_t, output_file=output_file,
                    )

INFO: Output files are stored in /home/sallen/MEOPAR/ANALYSIS/analysis-vicente/Ocean_Parcels/SHARED/particles_out.zarr.
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4320.0/4320.0 [00:22<00:00, 193.60it/s]


In [ ]:
delta_t * 21 / 3600

In [ ]:
21/3600*600